In [3]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 21.0 MB/s eta 0:00:00


In [4]:
import pandas as pd
import sqlite3
from pymongo import MongoClient


# 1. Cargar dataset

df = pd.read_csv('winemag-data-130k-v2.csv')


# 2. Renombrar columnas

df.rename(columns={
    'country': 'pais',
    'description': 'descripcion',
    'points': 'puntuacion',
    'price': 'precio'
}, inplace=True)


# 3. Crear nuevas columnas

# Mapeo de continentes
continente_dict = {
    'Italy': 'Europa', 'Portugal': 'Europa', 'US': 'America', 'Spain': 'Europa',
    'France': 'Europa', 'Germany': 'Europa', 'Argentina': 'America', 'Chile': 'America',
    'Austria': 'Europa', 'Australia': 'Oceania', 'South Africa': 'Africa',
    'New Zealand': 'Oceania', 'Israel': 'Asia', 'Brazil': 'America'
}
df['continente'] = df['pais'].map(continente_dict).fillna('Otros')

# Categoría de precio
def categoria_precio(precio):
    if pd.isna(precio):
        return 'Desconocido'
    elif precio < 20:
        return 'Barato'
    elif precio <= 50:
        return 'Medio'
    else:
        return 'Caro'
df['categoria_precio'] = df['precio'].apply(categoria_precio)

# Calidad según puntuación
def calidad(puntuacion):
    if puntuacion < 85:
        return 'Baja'
    elif puntuacion <= 90:
        return 'Media'
    else:
        return 'Alta'
df['calidad'] = df['puntuacion'].apply(calidad)


# 4. Generar Reportes

# Reporte 1: Promedio precio y puntuación por país
reporte1 = df.groupby('pais').agg(
    promedio_precio=('precio', 'mean'),
    promedio_puntuacion=('puntuacion', 'mean')
).sort_values(by='promedio_puntuacion', ascending=False)

# Reporte 2: Top 10 vinos más caros
reporte2 = df[['pais', 'precio', 'puntuacion', 'title']].sort_values(by='precio', ascending=False).head(10)

# Reporte 3: Cantidad de vinos por continente y categoría de precio
reporte3 = df.groupby(['continente', 'categoria_precio']).size().reset_index(name='cantidad')

# Reporte 4: Promedio de precio y cantidad de vinos por calidad
reporte4 = df.groupby('calidad').agg(
    promedio_precio=('precio', 'mean'),
    cantidad_vinos=('calidad', 'count')
).sort_values(by='promedio_precio', ascending=False)

# ========================
# 5. Exportar los reportes
# ========================
# Reporte 1 → CSV
reporte1.to_csv('reporte1_promedio_por_pais.csv', index=True)

# Reporte 2 → Excel
reporte2.to_excel('reporte2_top10_caros.xlsx', index=False)

# Reporte 3 → SQLite
conn = sqlite3.connect('reportes.db')
reporte3.to_sql('reporte3_vinos_por_categoria', conn, if_exists='replace', index=False)
conn.close()

# Reporte 4 → MongoDB
try:
    client = MongoClient('mongodb://localhost:27017/')
    db = client['reportes_vino']
    collection = db['reporte4_calidad']
    data = reporte4.reset_index().to_dict(orient='records')
    collection.insert_many(data)
    print("Reporte 4 guardado en MongoDB correctamente.")
except Exception as e:
    print("No se pudo conectar a MongoDB:", e)

print("Todos los reportes han sido generados y exportados correctamente.")


No se pudo conectar a MongoDB: localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 6885d1a6fd57480d3f9aa5f2, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>
Todos los reportes han sido generados y exportados correctamente.
